<a href="https://colab.research.google.com/github/abisubramanya27/CS6910_Assignment2/blob/main/partA/src/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mounting drive to store dataset
from google.colab import drive

drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

In [ ]:
%cd gdrive/MyDrive/assignments/cs6910/A2/Data
!pwd

In [ ]:
# !pip install split-folders

In [ ]:
# import splitfolders

# # Splitting the training data into training and validation set
# splitfolders.ratio('./inaturalist_12K/train', output='./inaturalist_12K/output', seed=1337, ratio=(.9, .1), group_prefix=None)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# Hyperparameters for building the model for Part-A
K_list_1 = [32, 32, 32, 32, 64, 64, 64, 64, 128, 128]       # List of number of filters in each non FC layer
F_list_1 = [11, 3, 5, 3, 3, 3, 3, 3, 3, 3]                  # List of size of filters in each non FC layer  
no_neurons_dense_1 = 1024                                   # Number of neurons in the dense FC layer
activation_fn_list_1 = ['relu']*6                           # List of activation function in each convolution and FC layer
P_list_1 = ['valid']*10                                     # List of padding options in each non FC layer ('valid' : no padding, 'same' : padding to make input and output same dimensions)
S_list_1 = [4, 2, 1, 1, 1, 1, 1, 2, 1, 1]                   # List of number of strides in each non FC layer
inp_img_shape_1 = (227, 227, 3)                             # Shape of input image from data
no_classes_1 = 10                                           # Number of output classes in the classification problem
BN_1 = True                                                 # True : Batch normalisation (BN) should be used, False : BN should not be used
dropout_p_1 = 0.3                                           # Probability of dropping out a neuron


def build_model_partA(inp_img_shape, K_list, F_list, no_neurons_dense, no_classes = 10, activation_fn_list = ['relu']*6, 
                      P_list = ['valid']*10, S_list = [1]*10, BN_yes = False, dropout_p = 0):
    '''
    Function to build the model comprising (5 conv+relu+maxpooling layers + 1 dense FC layer) for part A in keras
    Arguments :
        inp_img_shape -- shape of input image
        K_list -- List of number of filters in each non FC layer
        F_list -- List of size of filters (assumed same dimension in width and height) in each non FC layer  
        no_neurons_dense -- Number of neurons in the dense FC layer
        no_classes -- Number of output classes in the classification problem
        activation_fn_list -- List of activation function in each convolution and FC layer
        P_list -- List of padding options in each non FC layer 
                  ('valid' : no padding, 'same' : padding to make input and output same dimensions)
        S_list -- List of strides (assumed equal in width and height) in each non FC layer
        BN_yes -- True : Batch normalisation (BN) should be used, False : BN should not be used
        dropout_p -- Probability of dropping out a neuron
                     (The dropout is added for the single dense hidden layer alone after referring to many CNN architecture papers)

    Returns :
        model -- The keras sequential model of the CNN created
    '''
    model = Sequential()
    # First layer
    model.add(Conv2D(filters = K_list[0], kernel_size = (F_list[0], F_list[0]), strides = (S_list[0], S_list[0]), 
                     padding = P_list[0], input_shape = inp_img_shape))
    if BN_yes:
        model.add(BatchNormalization())
    model.add(Activation(activation_fn_list[0]))
    model.add(MaxPooling2D(pool_size=(F_list[1], F_list[1]), strides = (S_list[1], S_list[1]), padding = P_list[1]))

    # 4 Conv-relu-MaxPooling layers
    for l in range(1, 5):
        model.add(Conv2D(filters = K_list[2*l], kernel_size = (F_list[2*l], F_list[2*l]), strides = (S_list[2*l], S_list[2*l]), 
                         padding = P_list[2*l]))
        if BN_yes:
            model.add(BatchNormalization())
        model.add(Activation(activation_fn_list[l]))
        model.add(MaxPooling2D(pool_size = (F_list[2*l+1], F_list[2*l+1]), strides = (S_list[2*l+1], S_list[2*l+1]), padding = P_list[2*l+1]))
    
    # 1 dense FC layer
    model.add(Flatten())
    model.add(Dropout(dropout_p))
    model.add(Dense(units = no_neurons_dense))
    if BN_yes:
        model.add(BatchNormalization())
    model.add(Activation(activation = activation_fn_list[5]))

    # Output layer
    model.add(Dense(units = no_classes))
    if BN_yes:
        model.add(BatchNormalization())
    model.add(Activation(activation = 'softmax'))

    return model


# PART-A, Question 1 -- Building a model with (5 conv+relu+maxpooling layers + 1 dense FC layer) for image classification objective 
modelA = build_model_partA(inp_img_shape_1, K_list_1, F_list_1, no_neurons_dense_1, no_classes_1, 
                           activation_fn_list_1, P_list_1, S_list_1, BN_1, dropout_p_1)
modelA.summary()


In [ ]:
!pip install --upgrade wandb
!wandb login 6746f968d95eb71e281d6c7772a0469574430408

In [ ]:
import wandb
from wandb.keras import WandbCallback

run = wandb.init(project="assignment2", entity="abisheks", reinit=True, config={
                    "learning_rate": 1e-3,
                    "epochs": 10,
                    "batch_size": 32,
                    "loss_function": 'categorical_crossentropy',
                    "architecture": 'CNN',
                    "dataset": "iNaturalist_12K"
                })
config_1 = wandb.config

In [ ]:
# Generators for training, validation and test set image data for Part-A from the respective 
train_gen_1 = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 30,
    height_shift_range = 0.15,
    width_shift_range = 0.15,
    channel_shift_range = 10,
    shear_range = 0.15,
    zoom_range = 0.2,
    horizontal_flip = True).flow_from_directory('./inaturalist_12K/train', target_size = (227, 227), batch_size = config_1.batch_size)
val_gen_1 = ImageDataGenerator(
    rescale = 1./255).flow_from_directory('./inaturalist_12K/val', target_size = (227, 227), batch_size = config_1.batch_size)
test_gen_1 = ImageDataGenerator(
    rescale = 1./255).flow_from_directory('./inaturalist_12K/test', target_size = (227, 227), batch_size = config_1.batch_size)

In [ ]:
# Running sample run
tf.keras.backend.clear_session()
modelA.compile(optimizer = Adam(learning_rate=config.learning_rate), loss = config.loss_function, metrics = ['accuracy'])
modelA.fit(train_gen_1,
           epochs = config.epochs, 
           validation_data = val_gen_1,
           verbose = 2,
           callbacks = [WandbCallback()])
